In [1]:
import pandas as pd
import numpy as np

In [33]:
def Monte_GBM(S,v,mu,n,T, nstep=5):
    dt = T/nstep
    e = np.random.normal(loc=0.0, scale=1.0, size=(n,nstep))
    S_T = S*np.exp((mu - v ** 2 / 2) * dt + v * np.sqrt(dt) * e).cumprod(axis=1)
    return [np.mean(S_T[:,i]) for i in range(nstep)]

In [18]:
S = 100
K = 100
v = 0.25
mu = 0.05
n = 10000
T = 1

sim_res = Monte_GBM(S, v, mu, n, 1)

In [3]:
import yfinance as yf
from tqdm import tqdm

def fetch_stock_price(stock_symbol, start_date, end_date):
    # 使用 yf.Ticker() 建立 Ticker 對象
    stock = yf.Ticker(stock_symbol)

    # 使用 history() 方法取得歷史價格資訊
    stock_data = stock.history(start=start_date, end=end_date)

    return stock_data

stock_symbol = '2330.TW'

# 起始日期和結束日期
end_date = '2024-12-31'

# 擷取股票價格資訊
df = fetch_stock_price(stock_symbol=stock_symbol, start_date='2012-01-02',end_date=end_date)


In [19]:
df['ret'] = df['Close'].pct_change()
df = df.dropna()

In [34]:
Estimation_window = 30
total = 0
act = 0
n_step = 1

for i in tqdm(range(len(df)-Estimation_window+1)):
# for i in range(5):
    total+=1
    df_estimation = df.iloc[i:i+Estimation_window]
    drift = df_estimation['ret'].mean()
    vol = df_estimation['ret'].std()

    S_t = df_estimation['Close'][-1]

    df_test = df.iloc[i:i+Estimation_window+1]
    S_tp1 = df_test['Close'][-1]


    # pred dir
    pred = Monte_GBM(S = S_t, v=vol, mu=drift, n=10000, T=1, nstep=1)[0]
    dir_num = pred - S_t
    dir_dummy = 1 if dir_num > 0 else -1

    # act dir
    dir_num_act = S_tp1 - S_t
    dir_dummy_act = 1 if dir_num_act > 0 else -1

    if dir_dummy-dir_dummy_act==0:
        act+=1
    # print(f'pred : {pred}, dir : {dir_dummy}')
    # print(f'actual : {S_tp1}, dir : {dir_dummy_act}')
    # print('===================')
    # print()

100%|██████████| 2921/2921 [00:03<00:00, 885.36it/s]


In [35]:
act/total

0.4929818555289284

In [56]:
Estimation_window = 250
step_forward = 60

total = 0
act = 0


for i in tqdm(range(len(df)-Estimation_window+1)):
# for i in range(5):
    total+=1
    df_estimation = df.iloc[i:i+Estimation_window]
    # drift = np.log(df_estimation['ret']).mean()
    # vol = np.log(df_estimation['ret']).std()

    drift = df_estimation['ret'].mean()
    vol = df_estimation['ret'].std()
    S_t = df_estimation['Close'][-1]

    df_test = df.iloc[i:i+Estimation_window+step_forward]
    S_tp1 = df_test['Close'][-step_forward:].mean()


    # pred dir
    pred = np.mean(Monte_GBM(S = S_t, v=vol, mu=drift, n=10000, T=1, nstep=step_forward))
    dir_num = pred - S_t
    dir_dummy = 1 if dir_num > 0 else -1

    # act dir
    dir_num_act = S_tp1 - S_t
    dir_dummy_act = 1 if dir_num_act > 0 else -1

    if dir_dummy-dir_dummy_act==0:
        act+=1
    # print(f'pred : {pred}, dir : {dir_dummy}')
    # print(f'actual : {S_tp1}, dir : {dir_dummy_act}')
    # print('===================')
    # print()

100%|██████████| 2701/2701 [01:37<00:00, 27.64it/s]


In [52]:
# Estimation_window = 250
# step_forward = 20
# n=100000
# print(act/total)

0.5990373935579415


In [54]:
# Estimation_window = 250
# step_forward = 20
# n=100000
# print(act/total)

0.5994076268048871


In [57]:
# Estimation_window = 250
# step_forward = 60
# n=10000
print(act/total)

0.6638282117734172
